In [1]:
!pip install konlpy
from konlpy.tag import *
komoran = Komoran()

     |████████████████████████████████| 19.4MB 12.9MB/s 
     |████████████████████████████████| 92kB 14.0MB/s 
     |████████████████████████████████| 460kB 57.2MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [2]:
!pip install transformers boto3


     |████████████████████████████████| 1.8MB 4.2MB/s 
     |████████████████████████████████| 133kB 60.5MB/s 
     |████████████████████████████████| 2.9MB 60.9MB/s 
     |████████████████████████████████| 890kB 45.1MB/s 
     |████████████████████████████████| 71kB 11.0MB/s 
     |████████████████████████████████| 7.2MB 57.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=0c3cf3ad0498ce401d2020581cb3f512bc7390ed4ef05a42d38b4a249a06121d
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
ERROR: botocore 1.20.3 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.


In [ ]:
from torch.utils.data import TensorDataset, DataLoader
import torch
import pandas as pd

from transformers import BertConfig , BertForSequenceClassification
from transformers import BertTokenizer

model = BertForSequenceClassification.from_pretrained('beomi/kcbert-base',num_labels=7)
tokenizer = BertTokenizer.from_pretrained('beomi/kcbert-base')

In [179]:
import re
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
only_BMP_pattern = re.compile("["
        u"\U00010000-\U0010FFFF"  #BMP characters 이외
                           "]+", flags=re.UNICODE)
def split_token_get(text,tokenizer,MAX):
  split_tokens = []
  text = text.replace('\n',' ')
  text = emoji_pattern.sub(r'', text)
  text = only_BMP_pattern.sub(r'', text)
  
  no_punctuation = re.split(r'([!,?]+)|([.]+)|([,]+)|(["])|([\'])|([&]+)|([(]+)|([)]+)|([~]+)|([♡]+)|([☆,★]+)',
                              text.strip())
  no_punctuation_text = []

  for string in no_punctuation:
      if (string == '') or (string is None): continue
      no_punctuation_text.append(string)

  no_punctuation_text = ' '.join(no_punctuation_text)

  split_char = re.split(r'([ㄱ-ㅣ0-9]+)', no_punctuation_text.strip())
  split_char = ' '.join(split_char)
  split_char = re.split(r'([ㅎ]{2,})|([ㅜ,ㅠ]{2,})|([ㅗ]+)|([ㅋ,ㄱ,ㄲ]{2,})|\s+', split_char.strip())
  final_text = []
  for string in split_char:
        if (string == '') or (string is None): continue
        final_text.append(string)
    
  for token in komoran.morphs(text):
    for sub_token in tokenizer.tokenize(token):
      split_tokens.append(sub_token)
  if len(split_tokens) > MAX-2:
    split_tokens = split_tokens[:(MAX-2)]
  split_tokens = ["[CLS]"]+ split_tokens + ["[SEP]"]
  return split_tokens
def get_input_ids(token , tokenizer , MAX):
  segment_ids = [0] * len(token)
  input_ids = tokenizer.convert_tokens_to_ids(token)

  input_mask = [1] * (len(input_ids) -1 )
  input_mask += [0]
  
  padding = [0] * (MAX -len(input_ids) )

  input_ids += padding
  input_mask += padding
  segment_ids += padding

  assert len(input_ids) == MAX
  return input_ids 
def get_segment_ids(token, tokenizer, MAX):
  segment_ids = [0] * len(token)
  input_ids = tokenizer.convert_tokens_to_ids(token)
  padding = [0] * (MAX -len(input_ids) )
  
  segment_ids += padding
  assert len(segment_ids) == MAX
  return segment_ids 
def get_input_mask(token, tokenizer, MAX):
  
  input_ids = tokenizer.convert_tokens_to_ids(token)

  input_mask = [1]* (len(input_ids) -1 )
  input_mask += [0]

  padding = [0] * (MAX -len(input_ids) )

  input_mask += padding

  assert len(input_mask) == MAX
  return input_mask

In [5]:
import pandas as pd

data = pd.read_csv("labeled.csv")

In [7]:
for i in range(20):
  data = data.sample(frac=1)

In [10]:
sentences = data['Sentence']
labels = data['label'].values


sentences1 = sentences[32804:]
labels1 = labels[32804:]


sentences = sentences[:32804]
labels = labels[:32804]

In [11]:
tokenized_texts = [split_token_get(ss, tokenizer,128) for ss in sentences]
input_ids = [get_input_ids(ss,tokenizer,128)for ss in tokenized_texts]
segment_ids = [get_segment_ids(ss,tokenizer,128) for ss in tokenized_texts]
input_mask = [get_input_mask(ss,tokenizer,128) for ss in tokenized_texts]

input_ids = torch.tensor(input_ids)
segment_ids = torch.tensor(segment_ids)
input_mask = torch.tensor(input_mask)
labels = torch.tensor(labels,dtype = torch.long)

train_data = TensorDataset(input_ids,input_mask,labels)
train_dataloader = DataLoader(train_data, batch_size = 32)


tokenized_texts = [split_token_get(ss, tokenizer,128) for ss in sentences1]
input_ids = [get_input_ids(ss,tokenizer,128)for ss in tokenized_texts]
segment_ids = [get_segment_ids(ss,tokenizer,128) for ss in tokenized_texts]
input_mask = [get_input_mask(ss,tokenizer,128) for ss in tokenized_texts]

input_ids = torch.tensor(input_ids)
segment_ids = torch.tensor(segment_ids)
input_mask = torch.tensor(input_mask)
labels = torch.tensor(labels1,dtype = torch.long)

train_data1 = TensorDataset(input_ids,input_mask,labels)
train_dataloader1 = DataLoader(train_data1, batch_size = 8)

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()


if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')
model.cuda()

In [13]:

import numpy as np
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)
def format_time(elapsed):
    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
import time
import datetime
import random
import numpy as np
optimizer = AdamW(model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8)
epochs = 5
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)


model.zero_grad()
eval_accuracy = 0
nb_eval_steps = 0
for i in range(0, epochs):
  optimizer.zero_grad()
  t0 = time.time()
  total_loss = 0
  model.train()
  eval_accuracy = 0
  nb_eval_steps = 0
  for step,batch in enumerate(train_dataloader):
    model.train()
    if step%100 == 0 and not step ==0:
      elapsed = format_time(time.time() - t0)
      print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed) )
    
    batch = tuple(t.to(device) for t in batch)
    input_ids , input_mask , labels = batch

    outputs = model(input_ids= input_ids,
                    token_type_ids = None,
                    attention_mask=None,
                    labels = labels)
    loss = outputs

    total_loss += loss[0].item()

    loss[0].backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
  
    optimizer.step()
    scheduler.step()

    model.zero_grad()
  avg_train_loss = total_loss / len(train_dataloader)            

  print("")
  print("  Average training loss: {0:.2f}".format(avg_train_loss))
  print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
  
  t0 = time.time()
  model.eval()
  
  eval_accuracy = 0
  nb_eval_steps = 0
  for step,batch in enumerate(train_dataloader1):
    batch = tuple(t.to(device) for t in batch)

    input_ids , input_mask, labels = batch
    outputs = model(input_ids = input_ids ,
                    token_type_ids = None , 
                    attention_mask = input_mask 
                    )
    logits = outputs[0]

    logits = logits.detach().cpu().numpy()
    labels = labels.to('cpu').numpy()
    tmp_eval_accuracy = flat_accuracy(logits, labels)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1
  print("예측 Accuracy: {0:.5f}".format(eval_accuracy/nb_eval_steps))
  print("  Validation took: {:}".format(format_time(time.time() - t0)))


In [19]:
torch.save(model.state_dict(), '/content/drive/My Drive/today2.pt')

In [22]:
data = pd.read_csv("data.csv", encoding = 'cp949')

In [180]:
k = []
for i in range(0,100):
  name = '/content/youtube0_100/'
  path = ''
  dat = pd.read_csv(path+name)
  del dat['0']
  dat= dat.dropna(axis = 0)

  sss = dat['comment']
  tokenized_texts = [split_token_get(ss, tokenizer,128) for ss in sss]
  input_ids = [get_input_ids(ss,tokenizer,128)for ss in tokenized_texts]
  segment_ids = [get_segment_ids(ss,tokenizer,128) for ss in tokenized_texts]
  input_mask = [get_input_mask(ss,tokenizer,128) for ss in tokenized_texts]

  input_ids = torch.tensor(input_ids)
  segment_ids = torch.tensor(segment_ids)
  input_mask = torch.tensor(input_mask)

  train_data1 = TensorDataset(input_ids,input_mask)
  train_dataloader1 = DataLoader(train_data1, batch_size = 8)
  lab = [0,0,0,0,0,0,0]

  for step,batch in enumerate(train_dataloader1):
    
    batch = tuple(t.to(device) for t in batch)

    input_ids , input_mask = batch
    outputs = model(input_ids = input_ids ,
                    token_type_ids = None , 
                    attention_mask = input_mask 
                    )
    logits = outputs[0]
    for j in range(len(logits)):
      k.append(list(logits[j]).index(logits[j].max()))
  '''
    for j in range(len(logits)):
      num1 = list(logits[j]).index(logits[j].max())
      lab[num1] += 1

  ls[i] = lab
  print(lab)
  '''

In [175]:
dat['label'] =k

In [164]:
df = pd.DataFrame({'name':data['name'],'youtube_code':data['youtube_code'],'namu_code':data['namu_code'],
                   '0':np.array(ls).T[0],
                   '1':np.array(ls).T[1],
                   '2':np.array(ls).T[2],
                   '3':np.array(ls).T[3],
                   '4':np.array(ls).T[4],
                   '5':np.array(ls).T[5],
                   '6':np.array(ls).T[6]})

In [165]:
df.to_csv("d.csv")